# Genentech Cervical Cancer - ETL
https://www.kaggle.com/c/cervical-cancer-screening/

In [ ]:
# imports
import sys # for stderr
import numpy as np
import pandas as pd
import sklearn as skl
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# settings 
%logstop
%logstart  -o 'cc_feature_selection.log' rotate
plt.style.use('ggplot')
# constants
# plt.rcParams['figure.figsize'] = (10.0, 10.0)
# pd.set_option('display.max_rows', 50)
# pd.set_option('display.max_columns', 50)

In [ ]:
# versions 
import sys
print(pd.datetime.now())
print('Python: '+sys.version)
print('numpy: '+np.__version__)
print('pandas: '+pd.__version__)
print('sklearn: '+skl.__version__)

In [ ]:
from sqlalchemy import create_engine
engine = create_engine('postgresql://paulperry:@localhost:5432/ccancer')

In [ ]:
from pyace import ace

In [ ]:
import psycopg2
# import sqlite3

In [ ]:
db = psycopg2.connect("dbname='ccancer' user='paulperry' host='localhost' password=''")
#db = sqlite3.connect('../input/database.sqlite')

In [ ]:
patient_activity = pd.read_sql_query("select * from patient_activity limit 20000;", db)

In [ ]:
patient_activity[:10]

In [ ]:
patient_activity['date'] = pd.to_datetime(patient_activity.activity_year+
                                          patient_activity.activity_month,
                                          format='%Y%m')

In [ ]:
pat_min = patient_activity.groupby('patient_id').date.min()
pat_max = patient_activity.groupby('patient_id').date.max()
pat_count = patient_activity.groupby('patient_id').activity_type.count()

In [ ]:
pat_activity_count = patient_activity.groupby(['patient_id', 'activity_type']).activity_year.count()
pat_activity_count[:5]

In [ ]:
# pat_activity_count.loc[(slice(None),'A'),:].activity_year
pat_act = pat_activity_count.reset_index()
pat_act = pat_act.pivot('patient_id','activity_type')
pat_act = pat_act.activity_year

In [ ]:
pat_act.R.iloc[:5]

In [ ]:
pat_mo_delta =  ((pat_max - pat_min).dt.days / 30).astype(int)
pat_mo_delta[pat_mo_delta > 0].iloc[:5]

In [ ]:
patients_train = pd.read_sql_query("select * from patients_train limit 20000;", db)
patients_train.shape

In [ ]:
patients_train[:5]

In [ ]:
patients_train['date_min'] = pat_min
patients_train['date_max'] = pat_max
patients_train['date_delta'] = pat_mo_delta
patients_train['num_activity'] = pat_count
patients_train['type_A'] = pat_act.A
patients_train['type_R'] = pat_act.R

In [ ]:
patients_train

## Load 

In [ ]:
fdir = './features/'

In [ ]:
train_file = './input/patients_train.csv.gz'
train = pd.read_csv(train_file)
train.drop('patient_gender', axis=1, inplace=True)
train.set_index('patient_id', inplace=True)
train[:3]

## Run

In [ ]:
import datetime
start = datetime.datetime.now()
print(start)

In [ ]:
tab = pd.read_csv(fdir+'train_obgyn_drug.csv')
#tab.set_index('patient_id', inplace=True)
tab.shape

In [ ]:
tab.drop_duplicates(['patient_id','drug_id'], inplace=True)
tab.shape

In [ ]:
drugs = tab.drug_id.unique()
len(drugs)

In [ ]:
cat_cols = ['patient_age_group','patient_state','ethinicity','household_income','education_level']

In [ ]:
print(tab_pivot.shape)
tab_pivot[:10]

In [ ]:
tab_dummy = (tab_pivot[:] > 0).astype(int)

# tab_dummy[tab_dummy.iloc[:,0] > 0]

sums = tab_dummy.sum()
sums.sort_values(ascending=False, inplace=True)
tab_sub = tab_pivot.loc[:,sums[sums > 20].index] 

In [ ]:
step = 1000
for i in range(0,len(drugs),step):
    print(i, i+step, ':'), 
    subset = tab[tab.drug_id.isin(drugs[i:i+step])]
    tab_pivot = subset.pivot('patient_id','drug_id','drug_id')
    tab_dummy = (tab_pivot[:] > 0).astype(int)
    sums = tab_dummy.sum()
    sums.sort_values(ascending=False, inplace=True)
    tab_sub = tab_pivot.loc[:,sums[sums > 100].index]
    if tab_sub.shape[1] < 1:
        continue
    print(tab_sub.shape)
    dfall = pd.merge(train, tab_sub, left_index=True, right_index=True, how='left')
    ranks = ace(dfall, 'is_screener', cat_cols=[])
    df_ranks = pd.DataFrame(ranks, index=dfall.columns, columns=['ace','mean'])
    df_ranks = df_ranks.sort_values(by='ace', ascending=False)
    top_ranks = df_ranks[df_ranks.ace > 0]
    if top_ranks.shape[0] > 5:
        break
    gc.collect()

In [ ]:
tab_sub.shape

In [ ]:
nnn = 1

In [ ]:
break

In [ ]:
top_ranks.to_csv('drug_ranks_'+str(nnn)+'.csv')

In [ ]:
end   = datetime.datetime.now()
print('run time: '+str(end-start)+' at: '+str(end))

In [ ]:
break

In [ ]:
top_ranks = pd.read_csv('diagnosis_ranks_15.csv')
top_ranks.set_index('Unnamed: 0', inplace=True)
top_ranks[:10]

In [ ]:
qlist = list(top_ranks[top_ranks.ace > 0.003891].index)
for c in cat_cols: 
    qlist.remove(c)
qlist_str = "('"+qlist[0]+"'"
for c in qlist[1:]: 
    qlist_str=qlist_str+",'"+c+"'"
qlist_str=qlist_str+')'
qlist_str

In [ ]:
q = 'select * from diagnosis_code where diagnosis_code in '+qlist_str

diag_codes = pd.read_sql_query(q, engine)
diag_codes

In [ ]:
diag_codes.to_csv('diagnosis_top_codes.csv', mode='a', header=False)

In [ ]:
qlist

In [ ]:
tab[qlist].to_csv('diagnosis_top_'+str(nnn)+'.csv')

## physician codes

In [ ]:
physician = ['GO','GYN','OBG','OBS','OCC','ON','OTO','PCP','REN', 'FPP', 'PLN','FPG', 'FM', 'FP']
phy_codes = pd.read_csv('features/physician_specialty_codes.csv')

In [ ]:
phy_codes[phy_codes.specialty_code.isin(physician)]

In [ ]:
FP = FP , FM, PLN
OBG = OBG, GYN, REN, OBS
NOT = OTO, PCP, GO, FPP, OCC, FPG, ON 

In [ ]:
proc_years = pd.read_csv('./features/procedure/procedure_years.csv')
proc_years.shape

In [ ]:
proc_years[:10]

In [ ]:
proc_years.describe()

In [ ]:
proc_years.year_min.max()

In [ ]:
# avg_visits per year.

## OBG Exam

```
\copy (select t1.*, t2.specialty_code, t2.cbsa from diagnosis t1 join physician t2 on (t1.primary_practitioner_id=t2.physician_id) right join patients_train t3 on (t1.patient_id=t3.patient_id) where diagnosis_code = 'V70.0' and t2.specialty_code in ('OBG','GYN','REN', 'OBS')) to 'train_obgyn_exam.csv' with csv header
```

In [ ]:
q_obg_exam =  "select t1.*, t2.specialty_code, t2.cbsa from diagnosis t1 \
    join physician t2 on (t1.primary_practitioner_id=t2.physician_id) \
    right join patients_train t3 on (t1.patient_id=t3.patient_id) \
    where diagnosis_code = 'V70.0' and t2.specialty_code in ('OBG','GYN','REN', 'OBS'))"

train_obg_exam = .read_sql_query(q_obg_exam, db)

In [ ]:
q_fp_exam =  "select t1.*, t2.specialty_code, t2.cbsa from diagnosis t1 \
    join physician t2 on (t1.primary_practitioner_id=t2.physician_id) \
    right join patients_train t3 on (t1.patient_id=t3.patient_id) \
    where diagnosis_code = 'V70.0' and t2.specialty_code in ('FP', 'FM', 'PLN');"

train_fp_exam = pd.read_sql_query(q_fp_exam, db)
train_fp_exam.shape

In [ ]:
train_obg_exam = pd.read_csv(fdir+'train_obg_exam.csv')
train_obg_exam.shape

In [ ]:
train_fp_exam.shape

In [ ]:
# select t1.*, t2.specialty_code, t2.cbsa from diagnosis t1 \
# join physician t2 on (t1.primary_practitioner_id=t2.physician_id) \
# right join patients_train t3 on (t1.patient_id=t3.patient_id) \
# where t2.specialty_code in ('OBG','GYN','REN', 'OBS')

In [ ]:
train_obg = pd.read_csv(fdir+'train_obg.csv')
train_obg.shape

In [ ]:
train_obg[:10]

In [ ]:
[u'patient_id', u'diagnosis_date', u'diagnosis_code', 
 u'primary_practitioner_id',  u'primary_physician_role', u'specialty_code', u'cbsa']

In [ ]:
train_obg.specialty_code.value_counts()

In [ ]:
top_codes = train_obg.diagnosis_code.value_counts()
top_codes = top_codes.to_frame()
top_codes[:5]

In [ ]:
top_codes.columns = ['diagnosis_code','diag_count']
top
top_codes[:5]

In [ ]:
diagnosis_code = pd.read_sql_query('select * from diagnosis_code;', db)
diagnosis_code.shape

In [ ]:
diagnosis_code.set_index('diagnosis_code', inplace=True)
diagnosis_code[:5]

In [ ]:
top_codes = top_codes.join(diagnosis_code, rsuffix='_d')

In [ ]:
top_codes[:200].to_csv('top_gyn_diagnosis_codes.csv', index='diagnosis_code')

In [ ]:
patient1 = train_obg[train_obg.patient_id == 84548626]

In [ ]:
patient1.shape

In [ ]:
patient1.sort_values(by='claim_id')

In [ ]:
diagnosis_code.loc[patient1.diagnosis_code.unique()]

In [ ]:
obg_date_count[:4]

In [ ]:
obg_date_count = train_obg.groupby(['patient_id','diagnosis_date']).count()
obg_month_frequency = obg_date_count.reset_index()
obg_month_frequency = obg_month_frequency.groupby('patient_id').cbsa.count()
obg_month_frequency.name = 'obg_month_frequency'
obg_month_frequency[:10]

In [ ]:
obg_claim_count = train_obg.groupby(['patient_id','claim_id']).count()
obg_claim_count = obg_claim_count.reset_index()
obg_claim_count = obg_claim_count.groupby('patient_id').cbsa.count()
obg_claim_count.name = 'obg_claim_count'
obg_claim_count[:10]

In [ ]:
train_obg_practitioner = train_obg.groupby(['patient_id', 'primary_practitioner_id']).claim_id.count()
train_obg_practitioner.name = 'obg_practitioner_diagnosis_count'
train_obg_practitioner = train_obg_practitioner.to_frame()
train_obg_practitioner = train_obg_practitioner.reset_index(level=1)
train_obg_practitioner[:5]

In [ ]:
physician = train_obg[['primary_practitioner_id','specialty_code','cbsa']].copy()
physician.drop_duplicates(inplace=True)
physician.set_index('primary_practitioner_id')
physician[:5]

In [ ]:
train_obg_practitioner['screened'] = train.is_screener
train_obg_practitioner[:10]

In [ ]:
obg_screen_mean = train_obg_practitioner.groupby('primary_practitioner_id').screened.mean()
obg_screen_mean[:10]

In [ ]:
obg_screen_mean.name = 'obg_screen_pct'
obg_screen_mean[:5]

In [ ]:
train_obg_practitioner.shape

In [ ]:
obg_patient_count = train_obg_practitioner.groupby('primary_practitioner_id').screened.count()
obg_patient_count.name = 'obg_patient_count'
obg_patient_count[:10]

In [ ]:
obg_patient_count.describe()

In [ ]:
obg_patient_count[:20]

In [ ]:
# obg_patient_count.plot.bar(logy=True, use_index=False, sort_columns=True, figsize=(20,8))

In [ ]:
import gc
gc.collect()

In [ ]:
physician = physician.set_index('primary_practitioner_id')
physician[:5]

In [ ]:
physician = physician.join(obg_screen_mean)
physician = physician.join(obg_patient_count)
physician[:10]

In [ ]:
physician[:10]

In [ ]:
physician.to_csv('./features/obg_physician.csv')

In [ ]:
train_obg_practitioner[:7]

In [ ]:
train_obg_top_practitioner = train_obg_practitioner.reset_index()
# train_obg_top_practitioner = train_obg_top_practitioner.reset_index()
# train_obg_top_practitioner = train_obg_practitioner.groupby('patient_id')
train_obg_top_practitioner[:10]

In [ ]:
train_obg_top_practitioner = train_obg_top_practitioner.groupby('patient_id').max()
train_obg_top_practitioner[:5]

In [ ]:
train_obg_top_practitioner.shape

In [ ]:
train_obg_top_practitioner_full = train_obg_top_practitioner.join(physician, on='primary_practitioner_id')
train_obg_top_practitioner_full[:5]

In [ ]:
train_obg_top_practitioner_full[:20]

In [ ]:
train_obg_top_practitioner_full.drop('screened', axis=1, inplace=True)

In [ ]:
all_cols = {'cbsa': 'obg_cbsa',
            'primary_practitioner_id': 'obg_id',
           'obg_practitioner_diagnosis_count': 'obg_diagnosis_count',
           'specialty_code': 'obg_code'}

train_obg_top_practitioner_full.rename(columns=all_cols, inplace=True)
train_obg_top_practitioner_full[:2]

In [ ]:
bigone = train_obg_top_practitioner_full.join(obg_month_frequency)
bigone[:5]

In [ ]:
bigone = bigone.join(obg_claim_count)
bigone[:5]

In [ ]:
bigone['visited_obg'] = 1
bigone[:5]

In [ ]:
bigone.to_csv('./features/train_obg_pct.csv')

In [ ]:
bigone[:5]

In [ ]:
% pwd

In [ ]:
train_diagnosis = pd.read_csv(fdir+'train_diagnosis_top.csv.gz')
train_diagnosis.set_index('patient_id', inplace=True)
#    train_diagnosis[train_diagnosis > 0] = 1     # set dummies
  

In [ ]:
train_diagnosis[:10]

In [ ]:
train_diagnosis_dummies = pd.get_dummies(train_diagnosis)

In [ ]:
train_diagnosis_dummies[:10]

In [ ]:
train_diagnosis[train_diagnosis > 0] = 1

In [ ]:
obg = pd.read_csv(fdir+'test_obg.csv.gz')

In [ ]:
obg_date_count = obg.groupby(['patient_id','diagnosis_date']).count()
obg_month_frequency = obg_date_count.reset_index()
obg_month_frequency = obg_month_frequency.groupby('patient_id').cbsa.count()
obg_month_frequency.name = 'obg_month_frequency'

obg_claim_count = obg.groupby(['patient_id','claim_id']).count()
obg_claim_count = obg_claim_count.reset_index()
obg_claim_count = obg_claim_count.groupby('patient_id').cbsa.count()
obg_claim_count.name = 'obg_claim_count'

obg_practitioner = obg.groupby(['patient_id', 'primary_practitioner_id']).claim_id.count()
obg_practitioner.name = 'obg_practitioner_diagnosis_count'
obg_practitioner = obg_practitioner.to_frame()
obg_practitioner = obg_practitioner.reset_index(level=1)


In [ ]:
physician = pd.read_csv(fdir+'obg_physician.csv.gz')
physician = physician.set_index('primary_practitioner_id')

In [ ]:
obg_top_practitioner = obg_practitioner.reset_index()
obg_top_practitioner = obg_top_practitioner.groupby('patient_id').max()
obg_top_practitioner_full = obg_top_practitioner.join(physician, on='primary_practitioner_id')

all_cols = {'cbsa': 'obg_cbsa',
            'primary_practitioner_id': 'obg_id',
           'obg_practitioner_diagnosis_count': 'obg_diagnosis_count',
           'specialty_code': 'obg_code'}

obg_top_practitioner_full.rename(columns=all_cols, inplace=True)
obg_top_practitioner_full[:2]

bigone = obg_top_practitioner_full.join(obg_month_frequency)
bigone = bigone.join(obg_claim_count)
bigone['visited_obg'] = 1


In [ ]:
bigone[:10]

In [ ]:
bigone.to_csv(fdir+'test_obg_pct.csv')

In [ ]:
cancer_train=pd.read_csv('./features/train_diagnosis_cancer.csv.gz',index_col=None)

cancer_train['have_cancer']=1
cancer_train['sum_cancer']=1

idxC=cancer_train.groupby(['patient_id'])['patient_id'].mean().astype(int)
hc=cancer_train.groupby(['patient_id'])['have_cancer'].mean().astype(int)
sumhc=cancer_train.groupby(['patient_id'])['sum_cancer'].sum().astype(int)
cancer_train = pd.DataFrame({"patient_id": idxC.values, 'have_cancer': hc.values,'sum_cancer': sumhc.values })
print(cancer_train.head(5))

cancer_test=pd.read_csv('./features/test_diagnosis_cancer.csv.gz',index_col=None)
cancer_test['have_cancer']=1
cancer_test['sum_cancer']=1
print(cancer_test.info())
idxC=cancer_test.groupby(['patient_id'])['patient_id'].mean().astype(int)
hc=cancer_test.groupby(['patient_id'])['have_cancer'].mean().astype(int)
sumhc=cancer_test.groupby(['patient_id'])['sum_cancer'].sum().astype(int)
cancer_test = pd.DataFrame({"patient_id": idxC.values, 'have_cancer': hc.values,'sum_cancer': sumhc.values })


In [ ]:
cancer_train[['patient_id','have_cancer']][:5]

In [ ]:
cancer_test[['patient_id','have_cancer']][:5]

In [ ]:
cancer_train.to_csv('train_have_cancer.csv',index=False)
cancer_test.to_csv('test_have_cancer.csv',index=False)